In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


# Imports 

In [0]:
import cv2
import os 
import pandas as pd 
import numpy as np 
from keras.layers import Activation
from keras.layers import Lambda
from keras.layers import Conv2D
from keras.layers import Add
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import ZeroPadding2D
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.models import Model
import keras.backend as K
import tensorflow as tf
from google.colab.patches import cv2_imshow
#%tensorflow_version 1.x magic

In [0]:
dm  = (1280,320)
dm1 = (320,80)
dm2 = (160,40)
dm3 = (80,20)

In [0]:
def prepare_input(path = '/gdrive/My Drive/training/image_2'):
    tmp = [] 
    for item in os.listdir(path):
          tmp.append(item)
    tmp.sort()
    x = [] 
    for item in  tmp:
       tmpo = cv2.resize(cv2.imread(path+'/'+str(item)),dm)
       x.append(tmpo)
    X=x
    X=np.array(X)
    return X 

In [0]:
def prepare_label(path = '/gdrive/My Drive/training/gt_image_2'):
    tmp = [] 
    for item in os.listdir(path):
       tmp.append(item)
    tmp.sort()
    y = [] 
    for item in  tmp:
        tmpo = cv2.resize(cv2.imread(path+'/'+str(item)),dm)
        y.append(tmpo)
    Y=y
    Y=np.array(Y)   
    tmp1 = []
    tmp2 = [] 
    tmp3 = []
    for item in Y : 
      tmpo = item 
      tmp1.append(cv2.resize(tmpo,dm1))
      tmp2.append(cv2.resize(tmpo,dm2))
      tmp3.append(cv2.resize(tmpo,dm3))
    return tmp1,tmp2,tmp3

In [0]:
to_class=dict() 
to_color=dict() 
to_class[(0,0,0)]=0
to_class[(0,0,255)]=1
to_class[(255,0,255)]=2
to_color[0]=(0,0,0)
to_color[1]=(0,0,255)
to_color[2]=(255,0,255)

In [0]:
 def similar(x):
   a,b,c=x
   mn1=abs(0-a)+abs(0-b)+abs(0-c)
   mn2=abs(0-a)+abs(0-b)+abs(255-c)
   mn3=abs(255-a)+abs(0-b)+abs(255-c)
   if mn1<mn2 and mn1 < mn3 : 
     return 0
   elif mn2<mn1 and mn2<mn3: 
     return 1 
   else:
     return 2 
def encode_label(labels):
  val = np.zeros((np.shape(labels)[0],np.shape(labels)[1],np.shape(labels)[2],1))
  for i in range(np.shape(labels)[0]):
   for j in range(np.shape(labels)[1]):
       for k in range(np.shape(labels)[2]):
           if(tuple(labels[i][j][k])==(0,0,0)):
              val[i][j][k]=0
           elif(tuple(labels[i][j][k])==(0,0,255)):
              val[i][j][k]=1
           elif(tuple(labels[i][j][k])==(255,0,255)):
              val[i][j][k]=2
           else:
              val[i][j][k]=similar(tuple(labels[0][j][k]))
  return val



In [0]:
def build(width, height, n_classes, weights_path="/gdrive/My Drive/training/model.h5", train=False):
    inp = Input(shape=(height, width, 3))
    x = Lambda(lambda x: (x - 127.5)/255.0)(inp)

    # (1/2)
    y = Lambda(lambda x: tf.image.resize_bilinear(x, size=(int(x.shape[1])//2, int(x.shape[2])//2)), name='data_sub2')(x)
    y = Conv2D(32, 3, strides=2, padding='same', activation='relu', name='conv1_1_3x3_s2')(y)
    y = Conv2D(32, 3, padding='same', activation='relu', name='conv1_2_3x3')(y)
    y = Conv2D(64, 3, padding='same', activation='relu', name='conv1_3_3x3')(y)
    y_ = MaxPooling2D(pool_size=3, strides=2, name='pool1_3x3_s2')(y)
    y = Conv2D(128, 1, name='conv2_1_1x1_proj')(y_)

    y_ = Conv2D(32, 1, activation='relu', name='conv2_1_1x1_reduce')(y_)
    y_ = ZeroPadding2D(name='padding1')(y_)
    y_ = Conv2D(32, 3, activation='relu', name='conv2_1_3x3')(y_)
    y_ = Conv2D(128, 1, name='conv2_1_1x1_increase')(y_)
    y = Add(name='conv2_1')([y,y_])
    y_ = Activation('relu', name='conv2_1/relu')(y)

    y = Conv2D(32, 1, activation='relu', name='conv2_2_1x1_reduce')(y_)
    y = ZeroPadding2D(name='padding2')(y)
    y = Conv2D(32, 3, activation='relu', name='conv2_2_3x3')(y)
    y = Conv2D(128, 1, name='conv2_2_1x1_increase')(y)
    y = Add(name='conv2_2')([y,y_])
    y_ = Activation('relu', name='conv2_2/relu')(y)

    y = Conv2D(32, 1, activation='relu', name='conv2_3_1x1_reduce')(y_)
    y = ZeroPadding2D(name='padding3')(y)
    y = Conv2D(32, 3, activation='relu', name='conv2_3_3x3')(y)
    y = Conv2D(128, 1, name='conv2_3_1x1_increase')(y)
    y = Add(name='conv2_3')([y,y_])
    y_ = Activation('relu', name='conv2_3/relu')(y)

    y = Conv2D(256, 1, strides=2, name='conv3_1_1x1_proj')(y_)
    y_ = Conv2D(64, 1, strides=2, activation='relu', name='conv3_1_1x1_reduce')(y_) 
    y_ = ZeroPadding2D(name='padding4')(y_)
    y_ = Conv2D(64, 3, activation='relu', name='conv3_1_3x3')(y_)
    y_ = Conv2D(256, 1, name='conv3_1_1x1_increase')(y_)
    y = Add(name='conv3_1')([y,y_])
    z = Activation('relu', name='conv3_1/relu')(y)

    # (1/4)
    y_ = Lambda(lambda x: tf.image.resize_bilinear(x, size=(int(x.shape[1])//2, int(x.shape[2])//2)), name='conv3_1_sub4')(z)
    y = Conv2D(64, 1, activation='relu', name='conv3_2_1x1_reduce')(y_)
    y = ZeroPadding2D(name='padding5')(y)
    y = Conv2D(64, 3, activation='relu', name='conv3_2_3x3')(y)
    y = Conv2D(256, 1, name='conv3_2_1x1_increase')(y)
    y = Add(name='conv3_2')([y,y_])
    y_ = Activation('relu', name='conv3_2/relu')(y)

    y = Conv2D(64, 1, activation='relu', name='conv3_3_1x1_reduce')(y_)
    y = ZeroPadding2D(name='padding6')(y)
    y = Conv2D(64, 3, activation='relu', name='conv3_3_3x3')(y)
    y = Conv2D(256, 1, name='conv3_3_1x1_increase')(y)
    y = Add(name='conv3_3')([y,y_])
    y_ = Activation('relu', name='conv3_3/relu')(y)

    y = Conv2D(64, 1, activation='relu', name='conv3_4_1x1_reduce')(y_)
    y = ZeroPadding2D(name='padding7')(y)
    y = Conv2D(64, 3, activation='relu', name='conv3_4_3x3')(y)
    y = Conv2D(256, 1, name='conv3_4_1x1_increase')(y)
    y = Add(name='conv3_4')([y,y_])
    y_ = Activation('relu', name='conv3_4/relu')(y)

    y = Conv2D(512, 1, name='conv4_1_1x1_proj')(y_)
    y_ = Conv2D(128, 1, activation='relu', name='conv4_1_1x1_reduce')(y_)
    y_ = ZeroPadding2D(padding=2, name='padding8')(y_)
    y_ = Conv2D(128, 3, dilation_rate=2, activation='relu', name='conv4_1_3x3')(y_)
    y_ = Conv2D(512, 1, name='conv4_1_1x1_increase')(y_)
    y = Add(name='conv4_1')([y,y_])
    y_ = Activation('relu', name='conv4_1/relu')(y)

    y = Conv2D(128, 1, activation='relu', name='conv4_2_1x1_reduce')(y_)
    y = ZeroPadding2D(padding=2, name='padding9')(y)
    y = Conv2D(128, 3, dilation_rate=2, activation='relu', name='conv4_2_3x3')(y)
    y = Conv2D(512, 1, name='conv4_2_1x1_increase')(y)
    y = Add(name='conv4_2')([y,y_])
    y_ = Activation('relu', name='conv4_2/relu')(y)

    y = Conv2D(128, 1, activation='relu', name='conv4_3_1x1_reduce')(y_)
    y = ZeroPadding2D(padding=2, name='padding10')(y)
    y = Conv2D(128, 3, dilation_rate=2, activation='relu', name='conv4_3_3x3')(y)
    y = Conv2D(512, 1, name='conv4_3_1x1_increase')(y)
    y = Add(name='conv4_3')([y,y_])
    y_ = Activation('relu', name='conv4_3/relu')(y)

    y = Conv2D(128, 1, activation='relu', name='conv4_4_1x1_reduce')(y_)
    y = ZeroPadding2D(padding=2, name='padding11')(y)
    y = Conv2D(128, 3, dilation_rate=2, activation='relu', name='conv4_4_3x3')(y)
    y = Conv2D(512, 1, name='conv4_4_1x1_increase')(y)
    y = Add(name='conv4_4')([y,y_])
    y_ = Activation('relu', name='conv4_4/relu')(y)

    y = Conv2D(128, 1, activation='relu', name='conv4_5_1x1_reduce')(y_)
    y = ZeroPadding2D(padding=2, name='padding12')(y)
    y = Conv2D(128, 3, dilation_rate=2, activation='relu', name='conv4_5_3x3')(y)
    y = Conv2D(512, 1, name='conv4_5_1x1_increase')(y)
    y = Add(name='conv4_5')([y,y_])
    y_ = Activation('relu', name='conv4_5/relu')(y)

    y = Conv2D(128, 1, activation='relu', name='conv4_6_1x1_reduce')(y_)
    y = ZeroPadding2D(padding=2, name='padding13')(y)
    y = Conv2D(128, 3, dilation_rate=2, activation='relu', name='conv4_6_3x3')(y)
    y = Conv2D(512, 1, name='conv4_6_1x1_increase')(y)
    y = Add(name='conv4_6')([y,y_])
    y = Activation('relu', name='conv4_6/relu')(y)

    y_ = Conv2D(1024, 1, name='conv5_1_1x1_proj')(y)
    y = Conv2D(256, 1, activation='relu', name='conv5_1_1x1_reduce')(y)
    y = ZeroPadding2D(padding=4, name='padding14')(y)
    y = Conv2D(256, 3, dilation_rate=4, activation='relu', name='conv5_1_3x3')(y)
    y = Conv2D(1024, 1, name='conv5_1_1x1_increase')(y)
    y = Add(name='conv5_1')([y,y_])
    y_ = Activation('relu', name='conv5_1/relu')(y)

    y = Conv2D(256, 1, activation='relu', name='conv5_2_1x1_reduce')(y_)
    y = ZeroPadding2D(padding=4, name='padding15')(y)
    y = Conv2D(256, 3, dilation_rate=4, activation='relu', name='conv5_2_3x3')(y)
    y = Conv2D(1024, 1, name='conv5_2_1x1_increase')(y)
    y = Add(name='conv5_2')([y,y_])
    y_ = Activation('relu', name='conv5_2/relu')(y)

    y = Conv2D(256, 1, activation='relu', name='conv5_3_1x1_reduce')(y_)
    y = ZeroPadding2D(padding=4, name='padding16')(y)
    y = Conv2D(256, 3, dilation_rate=4, activation='relu', name='conv5_3_3x3')(y)
    y = Conv2D(1024, 1, name='conv5_3_1x1_increase')(y)
    y = Add(name='conv5_3')([y,y_])
    y = Activation('relu', name='conv5_3/relu')(y)

    h, w = y.shape[1:3].as_list()
    pool1 = AveragePooling2D(pool_size=(h,w), strides=(h,w), name='conv5_3_pool1')(y)
    pool1 = Lambda(lambda x: tf.image.resize_bilinear(x, size=(h,w)), name='conv5_3_pool1_interp')(pool1)
    pool2 = AveragePooling2D(pool_size=(h/2,w/2), strides=(h//2,w//2), name='conv5_3_pool2')(y)
    pool2 = Lambda(lambda x: tf.image.resize_bilinear(x, size=(h,w)), name='conv5_3_pool2_interp')(pool2)
    pool3 = AveragePooling2D(pool_size=(h/3,w/3), strides=(h//3,w//3), name='conv5_3_pool3')(y)
    pool3 = Lambda(lambda x: tf.image.resize_bilinear(x, size=(h,w)), name='conv5_3_pool3_interp')(pool3)
    pool6 = AveragePooling2D(pool_size=(h/4,w/4), strides=(h//4,w//4), name='conv5_3_pool6')(y)
    pool6 = Lambda(lambda x: tf.image.resize_bilinear(x, size=(h,w)), name='conv5_3_pool6_interp')(pool6)

    y = Add(name='conv5_3_sum')([y, pool1, pool2, pool3, pool6])
    y = Conv2D(256, 1, activation='relu', name='conv5_4_k1')(y)
    aux_1 = Lambda(lambda x: tf.image.resize_bilinear(x, size=(int(x.shape[1])*2, int(x.shape[2])*2)), name='conv5_4_interp')(y)
    y = ZeroPadding2D(padding=2, name='padding17')(aux_1)
    y = Conv2D(128, 3, dilation_rate=2, name='conv_sub4')(y)
    y_ = Conv2D(128, 1, name='conv3_1_sub2_proj')(z)
    y = Add(name='sub24_sum')([y,y_])
    y = Activation('relu', name='sub24_sum/relu')(y)

    aux_2 = Lambda(lambda x: tf.image.resize_bilinear(x, size=(int(x.shape[1])*2, int(x.shape[2])*2)), name='sub24_sum_interp')(y)
    y = ZeroPadding2D(padding=2, name='padding18')(aux_2)
    y_ = Conv2D(128, 3, dilation_rate=2, name='conv_sub2')(y)

    # (1)
    y = Conv2D(32, 3, strides=2, padding='same', activation='relu', name='conv1_sub1')(x)
    y = Conv2D(32, 3, strides=2, padding='same', activation='relu', name='conv2_sub1')(y)
    y = Conv2D(64, 3, strides=2, padding='same', activation='relu', name='conv3_sub1')(y)
    y = Conv2D(128, 1, name='conv3_sub1_proj')(y)

    y = Add(name='sub12_sum')([y,y_])
    y = Activation('relu', name='sub12_sum/relu')(y)
    y = Lambda(lambda x: tf.image.resize_bilinear(x, size=(int(x.shape[1])*2, int(x.shape[2])*2)), name='sub12_sum_interp')(y)

    out = Conv2D(n_classes, 1, activation='softmax', name='conv6_cls')(y)

    if train:
        aux_1 = Conv2D(n_classes, 1, activation='softmax', name='sub4_out')(aux_1)
        aux_2 = Conv2D(n_classes, 1, activation='softmax', name='sub24_out')(aux_2)

        model = Model(inputs=inp, outputs=[out, aux_2, aux_1])
    else:
        model = Model(inputs=inp, outputs=out)
        
    if weights_path is not None:
        model = load_model(weights_path,custom_objects={'tf': tf})
    return model

In [0]:
from keras import optimizers 
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)
m.compile(sgd, 'categorical_crossentropy', loss_weights=[1.0, 0.4, 0.16], metrics=['categorical_accuracy'])
m.fit(x=X,y=[tmp1,tmp2,tmp3],epochs=10000, callbacks=callbacks_list)

In [0]:
import time 
start_time = time.time()
new_model=build(width=1280,height=320,n_classes=3,train=False)
y = new_model.predict(X[0:30], batch_size=30)
duration = time.time() - start_time
#y = np.reshape(y[0],(290,80,320,3))
yy = [] 
for i in range(y.shape[0]):
   yy.append(cv2.resize(y[i],dm))
a = np.argmax(yy,axis=3)

In [0]:
from keras.utils import to_categorical
tmp1 = encode_label(tmp1)  
tmp2 = encode_label(tmp2)
tmp3 = encode_label(tmp3)
tmp1 = to_categorical(tmp1,3)
tmp2 = to_categorical(tmp2,3)
tmp3 = to_categorical(tmp3,3)

In [0]:
prd_list=[]
for ctr in range(290):
  prd = X[ctr]
  for i in range(np.shape(prd)[0]):
    for j in range(np.shape(prd)[1]):
        if a[ctr][i][j]==2:
           prd[i][j]=(0,255,0)
  prd_list.append(prd)
print(np.shape(prd_list))

(290, 320, 1280, 3)


In [0]:
def to_video(prd_list,path='/gdrive/My Drive/training/test_road.mp4'):
    out = cv2.VideoWriter(path,cv2.VideoWriter_fourcc(*'MP4V'), 7.0, (320,1280))
    for i in range(len(prd_list)):
        frame = prd_list[i]
        print(np.shape(frame))
        out.write(frame)
    out.release()
    cv2.destroyAllWindows()

In [0]:
def read_video(path):
    cap = cv2.VideoCapture(path)
    frames=[]
    while True: 
      ret , frame = cap.read() 
      if ret == False:
        break
      frames.append(frame)
    return frames    

In [0]:
def re_size(images):
     x = [] 
     for item in  images:
         tmpo = cv2.resize(item,dm)
         x.append(tmpo)
     X=x
     x=[]
     X=np.array(X)
     return X 

In [0]:
images = read_video('/gdrive/My Drive/road_car_view.mp4')

np.shape(images)

(2439, 720, 1280, 3)

In [0]:
sz = np.shape(images)
X = re_size(images)
np.shape(X)

(2439, 320, 1280, 3)

In [0]:
model = load_model("/gdrive/My Drive/training/model.h5",custom_objects={'tf': tf})













Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [0]:
#images=[]
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 1280, 3) 0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 320, 1280, 3) 0           input_1[0][0]                    
__________________________________________________________________________________________________
data_sub2 (Lambda)              (None, 160, 640, 3)  0           lambda_1[0][0]                   
__________________________________________________________________________________________________
conv1_1_3x3_s2 (Conv2D)         (None, 80, 320, 32)  896         data_sub2[0][0]                  
____________________________________________________________________________________________

In [0]:
def predict(y):
    
    yy = [] 
    for i in range(y.shape[0]):
        yy.append(cv2.resize(y[i],dm))
    y=[]
    a = np.argmax(yy,axis=3)
    
    yy=[]
    return a 

In [0]:
y = model.predict(X)
y = y[0]

In [0]:
y = predict(y[501:1500])

In [0]:
def paint(X,y) :
  prd_list=[] 
  for ctr in range(y.shape[0]):
    prd = X[ctr]
    for i in range(np.shape(prd)[0]):
      for j in range(np.shape(prd)[1]):
          if y[ctr][i][j]==2:
             prd[i][j]=(0,255,0)
    prd_list.append(prd)
  return prd_list

In [0]:
a = paint(X,y)

In [0]:
to_video(a)